# 2.3 Advanced Statistical Analysis with ANOVA

## 📚 Learning Objectives

By the end of this notebook, you will be able to:
- **Apply** one-way and two-way ANOVA to semiconductor manufacturing data
- **Design** and analyze factorial experiments for process optimization
- **Implement** Design of Experiments (DOE) principles
- **Interpret** interaction effects and their manufacturing implications
- **Validate** ANOVA assumptions and apply appropriate diagnostics
- **Calculate** effect sizes and assess practical significance
- **Perform** post-hoc comparisons with multiple testing corrections

## 🎯 What You'll Analyze

Using the **SECOM dataset** and carefully designed synthetic experiments, we'll explore:
1. **Tool comparison studies** with one-way ANOVA
2. **Multi-factor process optimization** with two-way ANOVA and interactions
3. **Factorial design analysis** for efficient screening
4. **Mixed-effects modeling** for hierarchical manufacturing data
5. **Response surface methodology** for process optimization

Let's start by loading our libraries and data.

In [ ]:
# Import essential libraries for statistical analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, shapiro, levene, kruskal
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.power import FTestAnovaPower
import warnings
from pathlib import Path

# Configure plotting and suppress warnings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("Set2")
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print("📊 Ready for advanced statistical analysis")

## 🏭 Loading and Preparing Data

We'll use the SECOM dataset as our foundation and create additional structured experimental data to demonstrate various ANOVA techniques. The SECOM data will serve as a base for tool comparison studies, while we'll generate factorial experimental data for DOE demonstrations.

In [ ]:
# Load SECOM data (using the same loader from 2.2)
def load_secom_for_anova():
    """Load and preprocess SECOM data for ANOVA analysis."""
    DATA_DIR = Path('../../datasets').resolve()
    SECOM_DATA_FILE = DATA_DIR / 'secom.data'
    SECOM_LABEL_FILE = DATA_DIR / 'secom_labels.data'
    
    if SECOM_DATA_FILE.exists() and SECOM_LABEL_FILE.exists():
        # Load real SECOM data
        X = pd.read_csv(SECOM_DATA_FILE, sep=" ", header=None, na_values='NaN')
        labels = pd.read_csv(SECOM_LABEL_FILE, sep=" ", header=None, na_values='NaN')
        
        # Basic preprocessing
        X = X.loc[:, X.notna().sum() > len(X) * 0.6]  # Keep columns with <40% missing
        X = X.fillna(X.median())  # Median imputation
        
        # Select representative features for analysis
        feature_variance = X.var()
        top_features = feature_variance.nlargest(20).index
        X_selected = X[top_features]
        
        # Add synthetic grouping variables for ANOVA
        n_samples = len(X_selected)
        np.random.seed(42)
        
        # Create tool assignments (4 tools)
        tool_assignments = np.random.choice(['Tool_A', 'Tool_B', 'Tool_C', 'Tool_D'], 
                                          size=n_samples, p=[0.3, 0.25, 0.25, 0.2])
        
        # Create lot assignments (simulate batch structure)
        n_lots = 20
        lot_size = n_samples // n_lots
        lot_assignments = np.repeat(range(n_lots), lot_size)[:n_samples]
        
        # Create final analysis dataset
        anova_data = pd.DataFrame({
            'tool': tool_assignments,
            'lot': lot_assignments,
            'quality_flag': labels.iloc[:, 0].values[:n_samples],
            'primary_response': X_selected.iloc[:, 0],  # Main response variable
            'secondary_response': X_selected.iloc[:, 1]  # Secondary response
        })
        
        return anova_data, X_selected
    else:
        # Generate synthetic data if SECOM not available
        print("🔄 SECOM data not found, generating synthetic semiconductor data...")
        return generate_synthetic_semiconductor_data()

def generate_synthetic_semiconductor_data():
    """Generate realistic synthetic semiconductor manufacturing data."""
    np.random.seed(42)
    n_samples = 800
    
    # Tool effects (different means for each tool)
    tool_effects = {'Tool_A': 0, 'Tool_B': 2.5, 'Tool_C': -1.2, 'Tool_D': 1.8}
    tools = list(tool_effects.keys())
    
    # Generate data with tool effects
    data_list = []
    for i, tool in enumerate(tools):
        n_tool = n_samples // 4
        base_response = np.random.normal(100 + tool_effects[tool], 5, n_tool)
        
        # Add some correlation structure
        secondary = base_response * 0.7 + np.random.normal(0, 2, n_tool)
        
        tool_data = pd.DataFrame({
            'tool': tool,
            'lot': np.random.randint(0, 20, n_tool),
            'quality_flag': np.random.choice([0, 1], n_tool, p=[0.93, 0.07]),
            'primary_response': base_response,
            'secondary_response': secondary
        })
        data_list.append(tool_data)
    
    anova_data = pd.concat(data_list, ignore_index=True)
    
    # Create feature matrix for consistency
    n_features = 20
    X_selected = pd.DataFrame(
        np.random.normal(0, 1, (n_samples, n_features)),
        columns=[f'feature_{i}' for i in range(n_features)]
    )
    
    return anova_data, X_selected

# Load the data
data, feature_matrix = load_secom_for_anova()
print(f"📦 Data loaded: {len(data)} samples")
print(f"🔧 Tools: {data['tool'].unique()}")
print(f"📊 Response variable range: {data['primary_response'].min():.2f} - {data['primary_response'].max():.2f}")

data.head()

## 📊 1. One-Way ANOVA: Tool Comparison Study

We'll start with a fundamental question in semiconductor manufacturing: **"Do different tools produce significantly different results?"**

This is a classic one-way ANOVA application where:
- **Factor**: Tool (categorical with 4 levels)
- **Response**: Primary process measurement (continuous)
- **Null Hypothesis**: All tools have the same mean response

In [ ]:
# Exploratory data analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Box plot by tool
sns.boxplot(data=data, x='tool', y='primary_response', ax=axes[0,0])
axes[0,0].set_title('Response Distribution by Tool')
axes[0,0].tick_params(axis='x', rotation=45)

# Histogram of overall response
axes[0,1].hist(data['primary_response'], bins=30, alpha=0.7, edgecolor='black')
axes[0,1].set_title('Overall Response Distribution')
axes[0,1].set_xlabel('Primary Response')

# Sample sizes by tool
tool_counts = data['tool'].value_counts()
axes[1,0].bar(tool_counts.index, tool_counts.values)
axes[1,0].set_title('Sample Size by Tool')
axes[1,0].tick_params(axis='x', rotation=45)

# Response vs tool (scatter with jitter)
for i, tool in enumerate(data['tool'].unique()):
    tool_data = data[data['tool'] == tool]
    x_jitter = np.random.normal(i, 0.1, len(tool_data))
    axes[1,1].scatter(x_jitter, tool_data['primary_response'], alpha=0.6, label=tool)

axes[1,1].set_xticks(range(len(data['tool'].unique())))
axes[1,1].set_xticklabels(data['tool'].unique())
axes[1,1].set_title('Individual Observations by Tool')
axes[1,1].legend()

plt.tight_layout()
plt.show()

# Descriptive statistics
print("📊 Descriptive Statistics by Tool:")
print(data.groupby('tool')['primary_response'].describe().round(3))

### Performing One-Way ANOVA

We'll use multiple approaches to conduct the one-way ANOVA and verify our results.

In [ ]:
# Method 1: Using scipy's f_oneway
tool_groups = [data[data['tool'] == tool]['primary_response'] for tool in data['tool'].unique()]
f_stat_scipy, p_value_scipy = f_oneway(*tool_groups)

print("🔬 One-Way ANOVA Results (scipy):")
print(f"F-statistic: {f_stat_scipy:.4f}")
print(f"p-value: {p_value_scipy:.6f}")
print(f"Significant at α=0.05: {'Yes' if p_value_scipy < 0.05 else 'No'}")

# Method 2: Using statsmodels (more detailed output)
model = ols('primary_response ~ C(tool)', data=data).fit()
anova_results = anova_lm(model, typ=2)

print("\n📈 Detailed ANOVA Table (statsmodels):")
print(anova_results)

# Calculate effect size (eta-squared)
ss_between = anova_results.loc['C(tool)', 'sum_sq']
ss_total = anova_results['sum_sq'].sum()
eta_squared = ss_between / ss_total

print(f"\n📊 Effect Size:")
print(f"Eta-squared (η²): {eta_squared:.4f}")

# Interpretation of effect size
if eta_squared < 0.01:
    effect_interpretation = "Small effect"
elif eta_squared < 0.06:
    effect_interpretation = "Medium effect"
else:
    effect_interpretation = "Large effect"

print(f"Effect size interpretation: {effect_interpretation}")

### ANOVA Assumption Checking

Before interpreting results, we must validate the key assumptions of ANOVA.

In [ ]:
# Extract residuals from the model
residuals = model.resid
fitted_values = model.fittedvalues

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Normality check - Q-Q plot
from scipy.stats import probplot
probplot(residuals, dist="norm", plot=axes[0,0])
axes[0,0].set_title("Q-Q Plot: Normality of Residuals")

# 2. Normality check - Histogram
axes[0,1].hist(residuals, bins=30, alpha=0.7, edgecolor='black')
axes[0,1].set_title("Histogram of Residuals")
axes[0,1].set_xlabel("Residuals")

# 3. Homoscedasticity check - Residuals vs Fitted
axes[0,2].scatter(fitted_values, residuals, alpha=0.6)
axes[0,2].axhline(y=0, color='red', linestyle='--')
axes[0,2].set_title("Residuals vs Fitted Values")
axes[0,2].set_xlabel("Fitted Values")
axes[0,2].set_ylabel("Residuals")

# 4. Boxplot of residuals by group
data_with_residuals = data.copy()
data_with_residuals['residuals'] = residuals
sns.boxplot(data=data_with_residuals, x='tool', y='residuals', ax=axes[1,0])
axes[1,0].set_title("Residuals by Tool")
axes[1,0].tick_params(axis='x', rotation=45)

# 5. Scale-Location plot
import numpy as np
sqrt_abs_residuals = np.sqrt(np.abs(residuals))
axes[1,1].scatter(fitted_values, sqrt_abs_residuals, alpha=0.6)
axes[1,1].set_title("Scale-Location Plot")
axes[1,1].set_xlabel("Fitted Values")
axes[1,1].set_ylabel("√|Residuals|")

# 6. Independence check - Residuals vs Order
axes[1,2].plot(residuals, alpha=0.6)
axes[1,2].axhline(y=0, color='red', linestyle='--')
axes[1,2].set_title("Residuals vs Observation Order")
axes[1,2].set_xlabel("Observation Order")
axes[1,2].set_ylabel("Residuals")

plt.tight_layout()
plt.show()

# Statistical tests for assumptions
print("🔍 Assumption Testing:")

# Shapiro-Wilk test for normality (use sample if data is large)
if len(residuals) > 5000:
    sample_residuals = np.random.choice(residuals, 5000, replace=False)
else:
    sample_residuals = residuals

shapiro_stat, shapiro_p = shapiro(sample_residuals)
print(f"Shapiro-Wilk normality test: W={shapiro_stat:.4f}, p={shapiro_p:.6f}")

# Levene's test for equal variances
levene_stat, levene_p = levene(*tool_groups)
print(f"Levene's test for equal variances: W={levene_stat:.4f}, p={levene_p:.6f}")

# Interpretation
print("\n📝 Assumption Interpretation:")
print(f"Normality: {'✅ Satisfied' if shapiro_p > 0.05 else '❌ Violated'} (p={shapiro_p:.6f})")
print(f"Equal variances: {'✅ Satisfied' if levene_p > 0.05 else '❌ Violated'} (p={levene_p:.6f})")

### Post-Hoc Analysis: Which Tools Differ?

When ANOVA is significant, we need post-hoc tests to identify which specific tools differ from each other.

In [ ]:
# Tukey's HSD (Honestly Significant Difference)
tukey_results = pairwise_tukeyhsd(data['primary_response'], data['tool'])
print("🎯 Tukey's HSD Post-Hoc Test:")
print(tukey_results)

# Create a more detailed comparison
mc = MultiComparison(data['primary_response'], data['tool'])
tukey_detailed = mc.tukeyhsd()

print("\n📊 Detailed Pairwise Comparisons:")
print(tukey_detailed.summary())

# Visualize the results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Confidence intervals for differences
tukey_detailed.plot_simultaneous(ax=ax1)
ax1.set_title("Tukey HSD: Simultaneous Confidence Intervals")

# Plot 2: Tool means with confidence intervals
tool_means = data.groupby('tool')['primary_response'].mean()
tool_stds = data.groupby('tool')['primary_response'].std()
tool_ns = data.groupby('tool')['primary_response'].count()
tool_ses = tool_stds / np.sqrt(tool_ns)

x_pos = range(len(tool_means))
ax2.bar(x_pos, tool_means, yerr=tool_ses*1.96, capsize=5, alpha=0.7)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(tool_means.index, rotation=45)
ax2.set_title("Tool Means with 95% Confidence Intervals")
ax2.set_ylabel("Primary Response")

plt.tight_layout()
plt.show()

# Summary of significant differences
print("\n🔍 Summary of Significant Differences:")
pairwise_data = tukey_detailed.summary().data[1:]  # Skip header
for row in pairwise_data:
    group1, group2, meandiff, p_adj, lower, upper, reject = row
    if reject:
        print(f"  {group1} vs {group2}: Mean difference = {meandiff:.3f}, p = {p_adj:.6f} ⭐")

## 📈 2. Two-Way ANOVA: Multi-Factor Analysis

Now we'll examine how two factors simultaneously affect our response. We'll create a factorial experiment examining the effects of tool and lot (representing different batches or time periods).

In [ ]:
# Create balanced subset for cleaner two-way ANOVA demonstration
# Select subset with balanced design (equal observations per cell)
lot_tool_counts = data.groupby(['lot', 'tool']).size().unstack(fill_value=0)
print("📊 Original Lot × Tool Counts:")
print(lot_tool_counts.head())

# For demonstration, let's create a more balanced dataset
# Select lots that have reasonable representation across tools
good_lots = lot_tool_counts[(lot_tool_counts > 5).all(axis=1)].index[:8]
balanced_data = data[data['lot'].isin(good_lots)].copy()

# Convert lot to categorical for cleaner analysis
balanced_data['lot'] = balanced_data['lot'].astype(str)

print(f"\n🎯 Balanced dataset: {len(balanced_data)} observations")
print("Tool × Lot combinations:")
print(pd.crosstab(balanced_data['tool'], balanced_data['lot']))

# Descriptive statistics
print("\n📊 Mean Response by Tool and Lot:")
pivot_means = balanced_data.pivot_table(values='primary_response', 
                                       index='tool', 
                                       columns='lot', 
                                       aggfunc='mean')
print(pivot_means.round(2))

In [ ]:
# Perform two-way ANOVA
model_2way = ols('primary_response ~ C(tool) + C(lot) + C(tool):C(lot)', 
                 data=balanced_data).fit()

anova_2way = anova_lm(model_2way, typ=2)
print("📊 Two-Way ANOVA Results:")
print(anova_2way)

# Calculate effect sizes for each factor
ss_tool = anova_2way.loc['C(tool)', 'sum_sq']
ss_lot = anova_2way.loc['C(lot)', 'sum_sq']
ss_interaction = anova_2way.loc['C(tool):C(lot)', 'sum_sq']
ss_total = anova_2way['sum_sq'].sum()

eta2_tool = ss_tool / ss_total
eta2_lot = ss_lot / ss_total
eta2_interaction = ss_interaction / ss_total

print(f"\n📈 Effect Sizes:")
print(f"Tool effect (η²): {eta2_tool:.4f}")
print(f"Lot effect (η²): {eta2_lot:.4f}")
print(f"Interaction effect (η²): {eta2_interaction:.4f}")

# Interpretation
print(f"\n🔍 Interpretation:")
print(f"Tool effect: {'Significant' if anova_2way.loc['C(tool)', 'PR(>F)'] < 0.05 else 'Not significant'}")
print(f"Lot effect: {'Significant' if anova_2way.loc['C(lot)', 'PR(>F)'] < 0.05 else 'Not significant'}")
print(f"Interaction: {'Significant' if anova_2way.loc['C(tool):C(lot)', 'PR(>F)'] < 0.05 else 'Not significant'}")

### Visualizing Interactions

Interaction plots help us understand how factors work together.

In [ ]:
# Create interaction plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Tool as x-axis, separate lines for each lot
tool_lot_means = balanced_data.groupby(['tool', 'lot'])['primary_response'].mean().unstack()

for lot in tool_lot_means.columns:
    axes[0].plot(tool_lot_means.index, tool_lot_means[lot], 
                marker='o', label=f'Lot {lot}')

axes[0].set_xlabel('Tool')
axes[0].set_ylabel('Mean Primary Response')
axes[0].set_title('Interaction Plot: Tool × Lot')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].grid(True, alpha=0.3)

# Plot 2: Lot as x-axis, separate lines for each tool
lot_tool_means = balanced_data.groupby(['lot', 'tool'])['primary_response'].mean().unstack()

for tool in lot_tool_means.columns:
    axes[1].plot(lot_tool_means.index, lot_tool_means[tool], 
                marker='o', label=tool)

axes[1].set_xlabel('Lot')
axes[1].set_ylabel('Mean Primary Response')
axes[1].set_title('Interaction Plot: Lot × Tool')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical interpretation of interaction
interaction_p = anova_2way.loc['C(tool):C(lot)', 'PR(>F)']
print(f"🔄 Interaction Analysis:")
print(f"p-value for interaction: {interaction_p:.6f}")

if interaction_p < 0.05:
    print("⚠️  Significant interaction detected!")
    print("   The effect of tool depends on the lot (or vice versa)")
    print("   Main effects should be interpreted cautiously")
else:
    print("✅ No significant interaction")
    print("   Main effects can be interpreted independently")

## 🧪 3. Factorial Design Analysis

Let's demonstrate a proper factorial design analysis using simulated experimental data. This shows how to efficiently study multiple factors and their interactions.

In [ ]:
# Generate a 2^3 factorial design for process optimization
# Factors: Temperature (Low/High), Pressure (Low/High), Flow Rate (Low/High)

def generate_factorial_experiment():
    """Generate a 2^3 factorial design with realistic semiconductor process data."""
    np.random.seed(123)
    
    # Design matrix (2^3 = 8 runs, with replication)
    from itertools import product
    
    factors = ['Temperature', 'Pressure', 'Flow_Rate']
    levels = [['Low', 'High']] * 3
    
    # Create full factorial design
    design_points = list(product(*levels))
    
    # Add replications (3 replicates per condition)
    n_reps = 3
    factorial_data = []
    
    for rep in range(n_reps):
        for i, (temp, press, flow) in enumerate(design_points):
            # Simulate realistic effects
            # Main effects
            temp_effect = 5 if temp == 'High' else 0
            press_effect = 3 if press == 'High' else 0
            flow_effect = 2 if flow == 'High' else 0
            
            # Interaction effects
            temp_press_effect = 2 if (temp == 'High' and press == 'High') else 0
            temp_flow_effect = -1 if (temp == 'High' and flow == 'High') else 0
            
            # Base response + effects + random error
            response = (85 + temp_effect + press_effect + flow_effect + 
                       temp_press_effect + temp_flow_effect + 
                       np.random.normal(0, 2))
            
            factorial_data.append({
                'Run': i + 1,
                'Rep': rep + 1,
                'Temperature': temp,
                'Pressure': press,
                'Flow_Rate': flow,
                'Yield': response
            })
    
    return pd.DataFrame(factorial_data)

factorial_df = generate_factorial_experiment()
print("🧪 Factorial Design Experiment (2³ with 3 replicates)")
print(f"Total runs: {len(factorial_df)}")
print("\nDesign matrix:")
print(factorial_df.groupby(['Temperature', 'Pressure', 'Flow_Rate'])['Yield'].mean().round(2))

# Display first few rows
print("\nFirst few experimental runs:")
print(factorial_df.head(10))

In [ ]:
# Analyze the factorial design
model_factorial = ols('Yield ~ C(Temperature) * C(Pressure) * C(Flow_Rate)', 
                     data=factorial_df).fit()

anova_factorial = anova_lm(model_factorial, typ=2)
print("🔬 Three-Factor Factorial ANOVA:")
print(anova_factorial)

# Calculate effect magnitudes
print("\n📊 Effect Magnitudes:")
effects = {}

# Extract coefficients (effects are half the coefficient values in coded designs)
coeffs = model_factorial.params
print("\nModel Coefficients:")
for param, coeff in coeffs.items():
    if param != 'Intercept':
        effects[param] = coeff
        significance = "***" if param in model_factorial.pvalues and model_factorial.pvalues[param] < 0.001 else \
                      "**" if param in model_factorial.pvalues and model_factorial.pvalues[param] < 0.01 else \
                      "*" if param in model_factorial.pvalues and model_factorial.pvalues[param] < 0.05 else ""
        print(f"{param}: {coeff:.3f} {significance}")

# Create effects plot (Pareto chart)
effect_names = []
effect_values = []
effect_pvalues = []

for param in model_factorial.params.index[1:]:  # Skip intercept
    if param in model_factorial.pvalues:
        effect_names.append(param.replace('C(', '').replace(')', '').replace('[T.High]', ''))
        effect_values.append(abs(model_factorial.params[param]))
        effect_pvalues.append(model_factorial.pvalues[param])

# Sort by effect magnitude
sorted_indices = np.argsort(effect_values)[::-1]
sorted_names = [effect_names[i] for i in sorted_indices]
sorted_values = [effect_values[i] for i in sorted_indices]
sorted_pvalues = [effect_pvalues[i] for i in sorted_indices]

# Plot effects
fig, ax = plt.subplots(figsize=(12, 8))
colors = ['red' if p < 0.05 else 'blue' for p in sorted_pvalues]
bars = ax.bar(range(len(sorted_names)), sorted_values, color=colors, alpha=0.7)

ax.set_xticks(range(len(sorted_names)))
ax.set_xticklabels(sorted_names, rotation=45, ha='right')
ax.set_ylabel('|Effect Size|')
ax.set_title('Pareto Chart of Effects (Red = Significant)')
ax.grid(True, alpha=0.3)

# Add significance line (arbitrary threshold for visualization)
significance_threshold = 1.5
ax.axhline(y=significance_threshold, color='orange', linestyle='--', 
           label=f'Practical significance threshold')
ax.legend()

plt.tight_layout()
plt.show()

## 📊 4. Advanced Analysis: Mixed Effects and Power Analysis

Let's explore more advanced topics including mixed-effects models for hierarchical data and power analysis for experiment planning.

In [ ]:
# Mixed-effects model: Tool as fixed effect, Lot as random effect
# This accounts for the hierarchical structure where lots are random samples

try:
    import statsmodels.formula.api as smf
    
    # Fit mixed-effects model
    mixed_model = smf.mixedlm("primary_response ~ C(tool)", 
                            data=data, 
                            groups=data["lot"]).fit()
    
    print("🏗️ Mixed-Effects Model Results:")
    print(mixed_model.summary())
    
    # Compare with fixed-effects ANOVA
    print("\n🔄 Model Comparison:")
    print(f"Fixed-effects AIC: {model.aic:.2f}")
    print(f"Mixed-effects AIC: {mixed_model.aic:.2f}")
    print(f"Better model: {'Mixed-effects' if mixed_model.aic < model.aic else 'Fixed-effects'}")
    
except ImportError:
    print("⚠️ Mixed-effects analysis requires additional packages")
    print("For production use, install: pip install statsmodels[mixed]")

# Power Analysis for Future Experiments
print("\n⚡ Power Analysis for Experiment Planning:")

def power_analysis_anova(effect_size, n_groups=4, alpha=0.05, power=0.80):
    """Calculate required sample size for ANOVA."""
    power_calc = FTestAnovaPower()
    
    # Calculate required sample size
    n_per_group = power_calc.solve_power(
        effect_size=effect_size,
        nobs=None,
        alpha=alpha,
        power=power,
        k_groups=n_groups
    )
    
    return int(np.ceil(n_per_group))

# Test different scenarios
effect_sizes = [0.1, 0.25, 0.4]  # Small, medium, large
effect_labels = ['Small', 'Medium', 'Large']

print("Required sample sizes per group:")
print("Effect Size | Power=0.80 | Power=0.90")
print("-" * 35)

for effect_size, label in zip(effect_sizes, effect_labels):
    n_80 = power_analysis_anova(effect_size, power=0.80)
    n_90 = power_analysis_anova(effect_size, power=0.90)
    print(f"{label:10} | {n_80:8} | {n_90:8}")

# Calculate achieved power from our current study
observed_f = f_stat_scipy
df_num = len(data['tool'].unique()) - 1
df_den = len(data) - len(data['tool'].unique())

# Estimate effect size from our data
n_per_group = len(data) / len(data['tool'].unique())
effect_size_observed = np.sqrt(eta_squared / (1 - eta_squared))

power_calc = FTestAnovaPower()
achieved_power = power_calc.solve_power(
    effect_size=effect_size_observed,
    nobs=n_per_group,
    alpha=0.05,
    power=None,
    k_groups=len(data['tool'].unique())
)

print(f"\n📊 Current Study Power Analysis:")
print(f"Observed effect size: {effect_size_observed:.3f}")
print(f"Achieved statistical power: {achieved_power:.3f}")
print(f"Power interpretation: {'Excellent' if achieved_power > 0.9 else 'Good' if achieved_power > 0.8 else 'Moderate' if achieved_power > 0.6 else 'Low'}")

## 🎯 5. Process Optimization Example

Let's demonstrate how to use ANOVA results for actual process optimization, including response surface methodology concepts.

In [ ]:
# Response surface analysis using factorial data
# This shows how to move from factorial screening to optimization

# Fit second-order model to factorial data (demonstration)
# In practice, you'd use Central Composite Design or Box-Behnken

# Convert categorical factors to numerical for RSM
factorial_df_numeric = factorial_df.copy()
factorial_df_numeric['Temp_coded'] = factorial_df_numeric['Temperature'].map({'Low': -1, 'High': 1})
factorial_df_numeric['Press_coded'] = factorial_df_numeric['Pressure'].map({'Low': -1, 'High': 1})
factorial_df_numeric['Flow_coded'] = factorial_df_numeric['Flow_Rate'].map({'Low': -1, 'High': 1})

# Fit model with interaction terms
rsm_model = ols('Yield ~ Temp_coded + Press_coded + Flow_coded + ' +
                'I(Temp_coded*Press_coded) + I(Temp_coded*Flow_coded) + I(Press_coded*Flow_coded)',
                data=factorial_df_numeric).fit()

print("🎯 Response Surface Model Summary:")
print(rsm_model.summary())

# Predict optimal conditions
# For this linear model, optimal is at the corners
temp_range = np.linspace(-1, 1, 10)
press_range = np.linspace(-1, 1, 10)

# Create prediction grid (fixing flow at high level)
temp_grid, press_grid = np.meshgrid(temp_range, press_range)
flow_fixed = 1  # High level

# Predict responses
predictions = []
for i in range(len(temp_range)):
    for j in range(len(press_range)):
        pred_data = pd.DataFrame({
            'Temp_coded': [temp_grid[j, i]],
            'Press_coded': [press_grid[j, i]], 
            'Flow_coded': [flow_fixed]
        })
        pred = rsm_model.predict(pred_data)
        predictions.append(pred[0])

predictions = np.array(predictions).reshape(temp_grid.shape)

# Visualize response surface
fig = plt.figure(figsize=(15, 5))

# Contour plot
ax1 = fig.add_subplot(131)
contour = ax1.contour(temp_grid, press_grid, predictions, levels=10)
ax1.clabel(contour, inline=True, fontsize=8)
ax1.set_xlabel('Temperature (coded)')
ax1.set_ylabel('Pressure (coded)')
ax1.set_title('Response Contours (Flow = High)')

# 3D surface plot
ax2 = fig.add_subplot(132, projection='3d')
surf = ax2.plot_surface(temp_grid, press_grid, predictions, alpha=0.7, cmap='viridis')
ax2.set_xlabel('Temperature')
ax2.set_ylabel('Pressure')
ax2.set_zlabel('Yield')
ax2.set_title('Response Surface')

# Optimization recommendations
ax3 = fig.add_subplot(133)
im = ax3.imshow(predictions, extent=[-1, 1, -1, 1], origin='lower', cmap='RdYlGn')
ax3.set_xlabel('Temperature (coded)')
ax3.set_ylabel('Pressure (coded)')
ax3.set_title('Optimization Heatmap')
plt.colorbar(im, ax=ax3, label='Predicted Yield')

# Mark optimal point
max_idx = np.unravel_index(np.argmax(predictions), predictions.shape)
optimal_temp = temp_grid[max_idx]
optimal_press = press_grid[max_idx]
ax3.plot(optimal_temp, optimal_press, 'r*', markersize=15, label='Optimum')
ax3.legend()

plt.tight_layout()
plt.show()

# Print optimization results
max_yield = np.max(predictions)
print(f"\n🎯 Optimization Results:")
print(f"Predicted maximum yield: {max_yield:.2f}")
print(f"Optimal temperature (coded): {optimal_temp:.2f}")
print(f"Optimal pressure (coded): {optimal_press:.2f}")
print(f"Optimal flow rate (coded): {flow_fixed}")

# Translate back to actual units (example)
print(f"\n🔧 Recommended Process Conditions:")
temp_actual = "High" if optimal_temp > 0 else "Low"
press_actual = "High" if optimal_press > 0 else "Low"
flow_actual = "High"
print(f"Temperature: {temp_actual}")
print(f"Pressure: {press_actual}")
print(f"Flow Rate: {flow_actual}")

## ✅ Summary and Manufacturing Insights

### Key Findings from Our Analysis

**Tool Comparison (One-Way ANOVA):**
- Significant differences between tools detected
- Effect size indicates practical importance
- Post-hoc tests identify specific tool pairs that differ
- Recommendations for tool matching or calibration

**Multi-Factor Analysis (Two-Way ANOVA):**
- Both tool and lot effects contribute to variation
- Interaction effects show how factors work together
- Hierarchical structure requires mixed-effects modeling

**Factorial Design:**
- Efficient screening of multiple factors
- Interaction effects revealed optimization opportunities
- Statistical significance vs. practical importance

**Process Optimization:**
- Response surface methodology guides improvement
- Optimal conditions identified through systematic experimentation
- Cost-benefit analysis of process changes

### 🚀 Best Practices for Manufacturing ANOVA

1. **Design Planning:**
   - Power analysis before experiments
   - Balanced designs when possible
   - Control for known sources of variation

2. **Assumption Validation:**
   - Always check normality, homoscedasticity, independence
   - Use robust alternatives when assumptions violated
   - Consider transformations for non-normal data

3. **Effect Size Reporting:**
   - Statistical significance ≠ practical importance
   - Report confidence intervals, not just p-values
   - Consider engineering tolerances and costs

4. **Multiple Comparisons:**
   - Control family-wise error rate
   - Use appropriate post-hoc procedures
   - Consider practical significance thresholds

### 🔜 Next Steps (Module 3.1)

In the next module, we'll extend these statistical foundations to **regression analysis for process engineers**, where we'll build predictive models and explore relationships between continuous variables.

### 📚 Additional Exercises

1. **Robustness Testing:** Apply Kruskal-Wallis test when normality is violated
2. **Sample Size Planning:** Calculate required samples for detecting 10% differences
3. **Cost Analysis:** Incorporate economic factors into optimization decisions
4. **Temporal Analysis:** Add time trends to mixed-effects models
5. **Multiple Responses:** Extend to MANOVA for multiple quality characteristics